# **Import Libraries and modules**

In [0]:
# https://keras.io/
!pip install -q keras
import keras

# New Section

In [0]:
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.datasets import mnist

### Load pre-shuffled MNIST data into train and test sets

In [0]:
(trainX, trainy), (testX, testy) = mnist.load_data()

### reshape dataset to have a single channel

In [243]:
width, height, channels = trainX.shape[1], trainX.shape[2], 1
print(width, height, channels)

28 28 1


In [0]:
trainX = trainX.reshape((trainX.shape[0], width, height, channels))
testX = testX.reshape((testX.shape[0], width, height, channels))


### Convert 1-dimensional class arrays to 10-dimensional class matrices


In [0]:
#hold original y in variable to figure out miss values later on
original_testy = testy

trainy = np_utils.to_categorical(trainy, 10)
testy = np_utils.to_categorical(testy, 10)

### report pixel means and standard deviations

In [246]:
print('Statistics train=%.3f (%.3f), test=%.3f (%.3f)' % (trainX.mean(), trainX.std(), testX.mean(), testX.std()))

Statistics train=33.318 (78.567), test=33.791 (79.172)


### create generator that centers pixel values

In [247]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen

### calculate the mean on the training dataset

In [248]:
datagen.fit(trainX)
print('Data Generator mean=%.3f, std=%.3f' % (datagen.mean, datagen.std))

Data Generator mean=33.318, std=78.567


### demonstrate effect on a single batch of samples

In [249]:
iterator = datagen.flow(trainX, trainy, batch_size=64)
iterator

### get a batch

In [0]:
batchX, batchy = iterator.next()

### pixel stats in the batch

In [251]:
print(batchX.shape, batchX.mean(), batchX.std())

(64, 28, 28, 1) -0.0026255564 0.9992786


### demonstrate effect on entire training dataset

In [252]:
iterator = datagen.flow(trainX, trainy, batch_size=len(trainX), shuffle=False)
iterator

In [0]:
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.regularizers import l2

model = Sequential()
# Extract edges and gradients features
model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu', input_shape=(28,28,1))) #26
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Trying to extract more features by increasing channels 
model.add(Convolution2D(16, (3, 3),  kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu')) #24
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Trying to extract more features by increasing channels 
model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #22

# reducing the size of parameters
model.add(MaxPooling2D(pool_size=(2, 2))) #11

# Since we have done  MP above we should try using 1x1 and fetch co dependend features.
model.add(Convolution2D(16, (1, 1), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #11
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Trying to Increase the channels to fetch parts of object
model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #9
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #7
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(16, (3, 3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu')) #5
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(16, (3,3))) #3
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Convolution2D(10, (3,3))) 

model.add(Flatten())
model.add(Activation('softmax'))

In [254]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_118 (Conv2D)          (None, 26, 26, 16)        160       
_________________________________________________________________
batch_normalization_92 (Batc (None, 26, 26, 16)        64        
_________________________________________________________________
dropout_92 (Dropout)         (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 24, 24, 16)        2320      
_________________________________________________________________
batch_normalization_93 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
dropout_93 (Dropout)         (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 22, 22, 16)        2320      
__________

### Train the model

In [255]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

def scheduler(epoch, lr):
  return round(0.003 * 1/(1 + 0.319 * epoch), 10)

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.003), metrics=['accuracy'])

# checkpoint

filepath="model_accuracy.best.hdf5"

reduce_LR = LearningRateScheduler(scheduler, verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#callbacks_list = [reduce_LR, checkpoint]

model.fit(trainX, trainy, batch_size=128, epochs=40, verbose=1, validation_data=(testX, testy), callbacks=[reduce_LR, checkpoint])

Train on 60000 samples, validate on 10000 samples
Epoch 1/40

Epoch 00001: LearningRateScheduler setting learning rate to 0.003.
60000/60000 [==============================] - 17s 284us/step - loss: 0.5894 - acc: 0.9289 - val_loss: 0.5941 - val_acc: 0.8199

Epoch 00001: val_acc improved from -inf to 0.81990, saving model to model_accuracy.best.hdf5
Epoch 2/40

Epoch 00002: LearningRateScheduler setting learning rate to 0.0022744503.
60000/60000 [==============================] - 8s 130us/step - loss: 0.1690 - acc: 0.9733 - val_loss: 0.1487 - val_acc: 0.9756

Epoch 00002: val_acc improved from 0.81990 to 0.97560, saving model to model_accuracy.best.hdf5
Epoch 3/40

Epoch 00003: LearningRateScheduler setting learning rate to 0.0018315018.
60000/60000 [==============================] - 8s 130us/step - loss: 0.1350 - acc: 0.9777 - val_loss: 0.1150 - val_acc: 0.9825

Epoch 00003: val_acc improved from 0.97560 to 0.98250, saving model to model_accuracy.best.hdf5
Epoch 4/40

Epoch 00004: Lear

### Wrongly predicted data

In [296]:
import numpy as np
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img

prediction = np.round(model.predict(testX))
wrong_pred = np.flatnonzero(testy != prediction)
print(wrong_pred)
#testy[9999]


ValueError: ignored

### Save Images

In [0]:
count = 0
for index in wrong_pred:
  if(count <25):
    #print(testX[index])
    #img_pil = array_to_img(testX[index])
    fileName = 'Wrong_Predicted_Image_' + str(index) + '.jpg'
    save_img(fileName, testX[index])
    #print(type(img_pil))
    count += 1

In [0]:
score = model.evaluate(testX, testy, verbose=0)

In [259]:
print(score)

[0.03563580178171396, 0.9931]


In [0]:
y_pred = model.predict(testX)

In [261]:
print(y_pred[:9])
print(testy[:9])

[[3.46519173e-13 5.60730529e-09 1.52785702e-07 1.64927783e-07
  1.20866884e-11 1.30508417e-11 1.15096182e-15 9.99999642e-01
  1.52352974e-11 9.83923254e-09]
 [1.39181418e-06 1.20113327e-06 9.99996185e-01 2.29654837e-08
  1.21730936e-09 1.67907008e-11 1.17422246e-06 6.68686551e-10
  9.12800147e-09 4.47066759e-11]
 [1.72814918e-08 9.99985456e-01 5.50655308e-08 2.20120988e-09
  6.83030339e-06 2.69551936e-08 1.72214655e-07 7.07868412e-06
  2.29320580e-08 4.60192666e-07]
 [9.99952793e-01 2.30607777e-09 4.71511339e-06 6.71190747e-08
  3.04885859e-08 1.55618110e-07 3.62793180e-05 7.83545318e-08
  5.43776059e-06 4.32432500e-07]
 [1.62186944e-10 6.50429710e-10 6.15694440e-11 9.30903900e-13
  9.99997377e-01 3.11955635e-11 1.96403130e-10 2.60728528e-09
  2.62515315e-10 2.61970149e-06]
 [3.20917359e-09 9.99948978e-01 5.07923081e-09 1.46807538e-10
  1.29080490e-05 5.06528020e-09 6.68927385e-08 3.70100024e-05
  2.62352096e-09 1.08256324e-06]
 [1.75892755e-11 1.32267280e-06 6.39218589e-09 4.49945275e